# Get some data from database
- display with some background highlighting
- also show graph

In [1]:
import getpass
import time
import pandas as pd
import cx_Oracle
from IPython.display import Markdown
from ipywidgets import interact_manual

# Stops warning about switching to SQL alchemy
import warnings
warnings.filterwarnings('ignore')

In [2]:
def run_query_df(sql, connstr, params=()):
    """Run query and return results as pandas dataframe"""
    conn = cx_Oracle.connect(connstr)
    df = pd.read_sql_query(sql, conn, params=params)#, chunksize=100)
    conn.close()  
    return df

def display_df(df):
    """Display full dataframe in Jupyter without auto truncation"""
    with pd.option_context('display.max_columns', None, 'display.max_rows', None, 'display.max_colwidth', -1):
        display(df)
        
def nowstring():
    return time.strftime("%d/%m/%Y %H:%M:%S")

### Connection details

In [3]:
ccr_username = "cclfdata"
ccr_database = "host.docker.internal"
ccr_password = getpass.getpass("CCR password: ")
ccr_connstr = "{}/{}@{}:1521/ccr".format(ccr_username, ccr_password, ccr_database)

CCR password:  ········


### Run the query
Press `Run interact` to refresh

In [4]:
@interact_manual
def run_supplier_query():
    sql = """
    select regi_region, count(*)
    from suppliers
    group by regi_region
    order by count(*)
    """
    supplier_df = run_query_df(sql, ccr_connstr)
    
    display(Markdown("## Supplier counts by region"))
    print(nowstring())
    # show tabulated results with some background styling
    display(supplier_df.style.background_gradient())
    # bar chart (rot rotates x-axis labels)
    supplier_df.plot.bar(x='REGI_REGION', y='COUNT(*)', rot=270)
    

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…